# A python implementation of $\mu$Kanren

$\mu$Kanren (microKanren)
is a minimalistic, yet complete, relational programming language, introduced as a stripped-down implementation of **miniKanren** (link al sito). 

What is a  *relational programming language*? In other programming paradigms, a program is a set of operations (functions, statements, etc.) which consume *inputs* to produce *outputs*. 
Instead, a relational program is a set of predicates on (logic) variables, without a distinction between inputs and outputs. Running a program means searching the values that can be assigned to logic variables so that those predicates hold. It does not return a value, but it enumerates solutions.

To me relational programming is a way to extract a usable and pure logical subset of prolog, and bring it to the masses.

Other people see it as DSLs for brute-force search.

The thing is that ??? this presentation is mind-blowing.

Where does it come from? In [reasoned_schemer], the authors introduced relational programming as a natural extension of functional programming. And so, guess what? the reference implementation of minikanren is in Scheme, which in turn does mean that the reference implementation of $\mu$Kanren  is in scheme.

Currently there are many implementation of the minikanren in many different languages; among them, the most succesful is probably [core.logic].

This post is a ipython notebook where I implement $\mu$Kanren and some syntactic sugar in python. You can download the original notebook here.

The first section of this post contains an implementation of $\mu$kanren in python; in the second section I introduce some syntactic sugar to make it more similar to miniKanren; the last section contains some examples of what
kind of programs can be written with it.

## $\mu$Kanren core

The objective of a $\mu$Kanren program is to find all the valid substitutions of logic variables so that all predicates are satisfied. 

### Logic variables

A `logic_variable` is an object. It is identified by an index: two `logic_variable`s are the same if they have the same index.

In [122]:
import collections

logic_variable = collections.namedtuple("logic_variable", ["index"])

def is_logic_var(x):
    return isinstance(x, logic_variable)

A counter, called the *fresh variable counter*, is used to generate unique indexes when creating new `logic_variable`s.

### Substitutions

From the [paper]
> A $\mu$Kanren program proceeds through the application of a **goal** to a **state**. Goals are often understood by analogy to predicates. Whereas the application of a predicate to an element of its domain can be either true or false, a goal pursued in a given state can either succeed or fail.

> A **state** is a pair of a substitution (represented as a dictionary) and a non-negative integer representing a fresh variable counter.

Substitutions are represented by python dictionaries whose keys $k$ are `logic_variable`s, and whose values $v$ are items that can replace that variable.

In [123]:
class SubstitutionMap(dict):
    pass

A `logic_variable` $X$ can be
*   free: the SubstitionMap does not contain $X$, and in that case predicates hold for every value it assumes. 
*   bound to another `logic_variable` $Y$: $X$ is equivalent to $Y$, and SubstitutionMap[$X$] = $Y$
*   bound to an item $i$: predicates hold when $X$ is replaced by $i$

We will see later what are the terms of the language, and so what are the items.

The **walk** method searches for a term's value. If the term is a value (i.e. not a logic variable), then it returns the value itself. Otherwise, it traverses the chain of substitutions looking to verify if a value has been already determined for that variable.

The **ext_s** is factory method that adds a new binding to an existing SubstitutionMap.

In [124]:
class SubstitutionMap(dict):
    
    def walk(self, var):
        while is_logic_var(var) and var in self:
            var = self[var]
        return var

    def ext_s(self, var, value):
        s = SubstitutionMap(self)
        s[var] = value
        return s

In [125]:
# test SubstitutionMap
x, y = logic_variable(0), logic_variable(1)

assert x == SubstitutionMap().walk(x), "x is free"

assert 1 == SubstitutionMap({x: 1}).walk(x), "x is bound to 1"

assert "something else" == SubstitutionMap({x: y})\
                           .ext_s(y, "something else")\
                           .walk(x), "walk must traverse the chain of substitutions"

The *fresh variable counter* is an int starting from 0, and it is used throught the evaluation to get new unique indexes for logic variables.

Initially, there are no substitutions, and the counter is 0. That state is called `empty_state`.

In [126]:
empty_state = (SubstitutionMap(), 0)

### The terms of the language: `unify`

The function `unify` defines which are the terms of the language.
It takes as arguments two objects, $x$ and $y$, and a SubstitutionMap *substitutions*.

Its objective is to add new bindings to *substitutions*, so that $x$ and $y$ becomes equivalent, i.e. 
```
unifying = unify(x, y, substitution)
unifying.walk(x) == unifying.walk(y)
```

If there is no way to get $x$ equivalent to $y$, for example because they are both already bound to terms
not equivalent, then it `unify` returns `None`. In this case we say that $x$ and $y$ do not unify under
`substitution` , i.e.

```
unify(x, y, substitution) == None
```

In this implementation, valid terms are python objects. At the beginning `unify` walks the two arguments
by using the SubstitutionMap. Then it checks the terms (i.e. the objects) it got. The two objects unify if
*   they are equals according to the `==` operator, and *substitution* stays unchanged
*   one is a `logic_variable` $v$, and in that case it means that $v$ is no more free, because it must be substituted with the other term to get the unification working
*   they are sequences, and in that case the unification is recursively applied term by term.

In [127]:
def unify(x, y, substitutions):
    if substitutions is None:
        return None

    x = substitutions.walk(x)
    y = substitutions.walk(y)

    if x == y:
        return substitutions
    elif is_logic_var(x):
        return substitutions.ext_s(x, y)
    elif is_logic_var(y):
        return substitutions.ext_s(y, x)
    elif is_sequence(x) and is_sequence(y):
        return unify_sequences(x, y, substitutions)
    return None

Note that I defined sequences as objects with the `__iter__` method. This is complitely arbitrary and independent from the rest of $\mu$Kanren.
To handle new kind of terms, we must update the `unify` function.

In [128]:
import itertools as it

def is_sequence(o):
    #  I want it to fail on strings
    return not is_logic_var(o) and hasattr(o, '__iter__')

def unify_sequences(xs, ys, substitutions):
    if len(xs) != len(ys):
        return None
    for a, b in it.izip(xs, ys):
        substitutions = unify(a, b, substitutions) 
    return substitutions

In [129]:
# Test unify
x, y = logic_variable(0), logic_variable(1)
assert unify(x, y, SubstitutionMap()) ==\
       SubstitutionMap({x: y}), "to unify x and y, substitute y to x"

assert unify(x, y, SubstitutionMap({x: "value x", y: "value y"}) ) is None, "they cannot be equivalent"
assert unify(x, 5, SubstitutionMap()) == SubstitutionMap({x: 5})
assert unify((x, 1, y), (1, 2, 3), SubstitutionMap()) is None, "sequences unify term-by-term"
assert unify((x, 1, y), (1, 1, 3), SubstitutionMap()) ==\
       SubstitutionMap({x: 1, y: 3}), "sequences unify term-by-term"

## Goals/Predicates builder

A *goal* is a function which takes as argument a state, i.e. a pair (`SubstitutionMap`, *fresh variable counter*), and returns a list of states which satisfy the goal.
In this implementation, *goal*s are generators yielding the valid states. If the generator is empty, then that goal
does not succeed.

A $\mu$Kanren program is built by using four goals builder: `equiv`, `call_fresh`, `disj` and `conj`.

### `Equiv`
 `equiv` builds a goal which succeeds if its two arguments unify, i.e. it yields the substitutions which make its arguments unify

In [130]:
def equiv(x, y):
    def _goal((substitutions, fresh_var_counter)):
        unifying = unify(x, y, substitutions)
        if unifying is not None:
            yield (unifying, fresh_var_counter)
    return _goal

In [131]:
# Test equiv
x, y = logic_variable(0), logic_variable(1)
assert list(equiv(x, 5)(empty_state)) == [(SubstitutionMap({x: 5}), 0)]
assert list(equiv(x, y)(empty_state)) == [(SubstitutionMap({x: y}), 0)]

### `call_fresh`
The call/fresh goal constructor creates a new `logic_variable`.

It takes as argument a unary function $f$, which must return a goal, and it returns
a new goal which runs $f$ by binding its argument to a new `logic_variable`.
It leaves unchanged the substitutions, but it increments the *fresh variable counter*,
to ensure unicity of variables indexes.

In [132]:
def call_fresh(f):
    def _new_goal((substitutions, fresh_var_counter)):
        return f(logic_variable(fresh_var_counter))((substitutions, fresh_var_counter+1))
    return _new_goal


As shown in the following tests, if I want to introduce a new logic variable in a goal $G$:
*   I wrap the goal $G$ in a unary function $f$
*   I use the sole argument of $f$ as logic_variable in $G$
*   I pass $f$ to `call_fresh` and I use the resulting goal

This is quiet verbose: the API will be simplified with the operator `fresh`. Since this is not
part of the core of $\mu$Kanren, we will see that later.

In [133]:
# test call_fresh
x, y = logic_variable(0), logic_variable(1)
def is_five(new_var):
    return equiv(new_var, 5)
goal = call_fresh(is_five)
assert list(goal(empty_state)) ==\
       [(SubstitutionMap({x: 5}), 1)], "x must be 5"

# for every new variable, I need a new call_fresh
def f(var):
    def _g(other_var):
        return equiv(var, other_var)
    return call_fresh(_g)
goal = call_fresh(f)

assert list(goal(empty_state)) ==\
       [(SubstitutionMap({x: y}), 2)], "x and y are equivalent but not bound to a term"


### `disj`

`disj` takes as arguments some goals, and it returns a new goal which succeeds for a given state $s$ if either succeeds for that state $s$.

I use `ANY` as synonim of `disj`, since it is equivalent to the built-in function `any`.

In [134]:
def disj(*goals):
    def _new_goal(state):
        return it.chain.from_iterable(g(state) for g in goals)
    return _new_goal

ANY = disj

In [196]:
# test disj
x, y = logic_variable(0), logic_variable(1)

def g_any_ok(var_x, var_y):
    """
    It succeeds if x is 2, or y is 3 or x is y
    """
    return ANY(equiv(var_x, 2),
               equiv(var_y, 3),
               equiv(var_x, var_y))

## 2 variables, so 2 call_fresh(unary function)
r = list(call_fresh(lambda x: call_fresh(lambda y: g_any_ok(x, y)))(empty_state))
assert len(r) == 3 \
        and (SubstitutionMap({x: 2}), 2) in r \
        and (SubstitutionMap({y: 3}), 2) in r\
        and ({x: y}, 2) in r, "Three solutions must be returned"

### `conj`
`conj` takes as arguments some goals, and it returns a new goal which succeeds for a given state $s$
if all of them succeed for that state $s$. 

I use `ALL` as synonim of `conj`, since it is equivalent to the built-in function `all`.

In [197]:
def make_stream(goal, s):
    return it.chain.from_iterable(it.imap(goal, s))

def conj(*goals):
    """
    It returns a goal which succeeds if all the goals passed as argument succeed
    for that state.
    """
    def _new_goal(state):
        stream = goals[0](state)
        for g in goals[1:]:
            stream = make_stream(g, stream)
        return stream
    return _new_goal

ALL = conj

In [198]:
# test disj
x, y = logic_variable(0), logic_variable(1)

def g_all_ko(var_x, var_y):
    """
    It succeeds if x is 2, or y is 3 or x is y
    """
    return ALL(equiv(var_x, 2),
               equiv(var_y, 3),
               equiv(var_x, var_y))

# fresh will fix this mess
r = list(call_fresh(lambda x: call_fresh(lambda y: g_all_ko(x, y)))(empty_state))
assert len(r) == 0, "x is not equivalent to y"

def g_all_ok(var_x, var_y):
    """
    It succeeds if x is 2 or 3, and y is 3 and y is x
    """
    return ALL(ANY(equiv(var_x, 2), equiv(var_x, 3)),
               equiv(var_y, 3),
               equiv(var_x, var_y))

r = list(call_fresh(lambda x: call_fresh(lambda y: g_all_ok(x, y)))(empty_state))
assert r == [(SubstitutionMap({x: 3, y: 3}), 2)], "x cannot be 2"

**That's it**. This is the core of $\mu$Kanren.

In the next session we will add some syntactic sugar from minikanren.

## Minikanren sugar

Now that we have the core of the system, it is time to add some sugar toe make it pleasant to use.

### `fresh`
`fresh` is a `call_fresh` without the single-variable limitation.

In [199]:
import inspect

def fresh_helper(curried_goal, nargs):
    if nargs <= 1:
        return call_fresh(curried_goal)
    else:
        return call_fresh(lambda x: fresh_helper(p(curried_goal, x), nargs-1))
    
def fresh(variadic_goal):
    # Discover now many arguments it has, and apply the pattern
    # wrap in a function -> pass it to call_fresh
    positional_args = inspect.getargspec(variadic_goal).args
    return fresh_helper(variadic_goal, len(positional_args))

In [200]:
# test fresh         
# I use here the same functions used to test ALL. The API here is soooo much better
assert list(fresh(g_all_ko)(empty_state)) == []
assert list(fresh(g_all_ok)(empty_state)) == \
       [(SubstitutionMap({x: 3, y: 3}), 2)], "x cannot be 2"

### `run`

`run` calls `fresh` for us. It takes a goal as arguments and yield the substitutions
for which that goal succeeds.

It hides `fresh` and the *fresh variable counter*.
In the next section we will redefine it to hide also the `SubstitionMap`.

In [201]:
def run(goal_builder):
    for s, _ in fresh(goal_builder)(empty_state):
        yield s
        
list(run(g_any_ok))

[{logic_variable(index=0): 2},
 {logic_variable(index=1): 3},
 {logic_variable(index=0): logic_variable(index=1)}]

### `reify`

`reify` translates the internal representation of variables
to a human readable form.

The reification starts by `walk`ing a variable in the `SubstitutionMap`.

If the result is a `logic_variable`, then that variable could be free,
or not bound to an item but equivalent to another `logic_variable`.
To represent this, I use the objects `any_value`. They contain an id,
and two `any_value`s are equivalent if that id is the same.

Sequences are reified to lists, where each element is the reified version
of the elements of original sequence.

Items are reified to themselves.

In [202]:
any_value = collections.namedtuple("any_value", ["id"])

def reify_sequence(subs, objects):
    return [reify(subs, o) for o in objects]

def reify(substitutions, o):
    o = substitutions.walk(o)
    if is_logic_var(o):
        return any_value(o.index)
    elif is_sequence(o):
        return reify_sequence(substitutions, o)
    else:
        return o

In [203]:
# test reify
x, y, z = logic_variable(0), logic_variable(1), logic_variable(2)
empty_sm = SubstitutionMap()
test_sm = SubstitutionMap({x: 2, y: z, z: 3})

assert any_value(id=0) == reify(empty_sm, x), "x is a free variable"
assert 3  == reify(test_sm, z) == reify(test_sm, y), "y and z walk to 3"
assert reify(test_sm, logic_variable(12)) != \
       reify(test_sm, logic_variable(logic_variable(2))), "free variables not equivalent"

### `run`, reworked

I let `run` handle the reification, and hide completely the `SubstitutionMap`.

The new `run` takes as argument a variadic function building a goal,
it calls `fresh` to create `logic_variable`s, and it yields all the solutions which make the goal
succeeds. A solution is a tuple containing the reified variables in the same order of the goal builder arguments.

I exploit the fact that variable indexes match the order of the arguments they are bound to.

In [211]:
def run(goal_builder, stop=None):
    args = inspect.getargspec(goal_builder).args
    var_indexes = range(len(args))
    substitutions = fresh(goal_builder)(empty_state)
    for s, _ in it.islice(substitutions, stop):
        # it reifies the variables in the order of corresponding
        # arguments of goal
        yield tuple(reify(s, logic_variable(idx)) for idx in var_indexes)

In [212]:
# test run

solutions = list(sorted(run(g_any_ok)))
assert solutions == [(2, any_value(id=1)), # g_any_ok(2, whatever)
                     (any_value(id=0), 3), # g_any_ok(whatever, 3)
                     (any_value(id=1), any_value(id=1))] # g_any_ok(whatever, the same)

Examples
Querying Game of Thrones

In [102]:
got_characters = {0: ('catelyn', 'tully'),
                  1: ('eddard', 'stark'),
                  2: ('sansa', 'stark'),
                  3: ('benjen', 'stark'),
                  4: ('robb', 'stark'),
                  5: ('joffrey', 'baratheon'),
                  6: ('stannis', 'baratheon'),
                  7: ('cersei', 'lannister'),
                  8: ('tyrion', 'lannister'),
                  9: ('tommen', 'baratheon'),
                  10: ('jon', 'snow'),
                  11: ('myrcella', 'baratheon'),
                  12: ('tywin', 'lannister'),
                  13: ('jaime', 'lannister'),
                  14: ('rickon', 'stark'),
                  15: ('arya', 'stark'),
                  16: ('brandon', 'stark'),
                  17: ('renly', 'baratheon'),
                  18: ('robert', 'baratheon')}

In [103]:
def charactero(characterid, name, surname):
    return ANY(
        *[ALL(equiv(characterid, k), equiv(name, n), equiv(surname, s))
          for (k, (n, s)) in got_characters.iteritems()]
    )

In [104]:
list(run(
        lambda x, y: charactero(1, x, y)
    ))

[('stark', 'eddard')]

In [105]:
list(run(
        lambda x, y: charactero(x, y, 'baratheon')
    ))

[('joffrey', 5),
 ('stannis', 6),
 ('tommen', 9),
 ('myrcella', 11),
 ('renly', 17),
 ('robert', 18)]

In [106]:
got_houses = {'stark': [0, 1, 2, 3, 4, 10,  15, 16],
              'tully': [0],
              'lannister': [5, 7, 8, 9, 11, 12, 13],
              'baratheon': [5, 7, 9, 11, 6, 18]}

def _houseo(house_name, characterid):
    g = ANY(
        *[ALL(equiv(house_name, k), equiv(characterid, v))
          for (k, vs) in got_houses.iteritems() for v in vs ])
    return g

In [58]:
list(run(
        lambda i: _houseo('baratheon', i)
    ))

[(5,), (7,), (9,), (11,), (6,), (18,)]

In [59]:
def houseo(family_name, name, surname):
    def _f(characterid):
        return ALL(
            _houseo(family_name, characterid),
            charactero(characterid, name, surname)
        )
    return fresh(_f)

In [60]:
list(run(
        lambda house, surname: houseo(house, 'joffrey', surname)
    ))

[('baratheon', 'baratheon'), ('lannister', 'baratheon')]

In [61]:
list(run(
        lambda house, name: houseo(house, name, house)
    ))

# no jon snow

[('baratheon', 'joffrey'),
 ('baratheon', 'tommen'),
 ('baratheon', 'myrcella'),
 ('baratheon', 'stannis'),
 ('baratheon', 'robert'),
 ('lannister', 'cersei'),
 ('lannister', 'tyrion'),
 ('lannister', 'tywin'),
 ('lannister', 'jaime'),
 ('stark', 'eddard'),
 ('stark', 'sansa'),
 ('stark', 'benjen'),
 ('stark', 'robb'),
 ('stark', 'arya'),
 ('stark', 'brandon'),
 ('tully', 'catelyn')]

In [62]:
def not_equiv(x, y):
    def _goal(state):
        substitutions, counter = state
        unified = unify(x, y, substitutions)
        if unified is None:
            yield (substitutions, counter)
    return _goal

list(run(
        lambda house, name, family: ALL(
            houseo(house, name, family),
            not_equiv(house, family))
    ))

[('baratheon', 'cersei', 'lannister'),
 ('lannister', 'joffrey', 'baratheon'),
 ('lannister', 'tommen', 'baratheon'),
 ('lannister', 'myrcella', 'baratheon'),
 ('stark', 'catelyn', 'tully'),
 ('stark', 'jon', 'snow')]

Data structures

cons

In [63]:
def emptyo(d):
    return equiv(d, ())

def conso(a, b, cons):
    return equiv((a, b), cons)

def firsto(cons, elt):
    return fresh(lambda t: conso(elt, t, cons))

def tailo(cons, tail):
    return fresh(lambda first: conso(first, tail, cons))

In [64]:
list(run(
        lambda x: firsto(x, 1)))

[([1, any_value(id=1)],)]

In [85]:
def consify(lst):
    cons = ()
    for i in reversed(lst):
        cons = (i, cons)
    return cons

def ideconsify(cons):
    while cons:
        car, cdr = cons
        yield car
        cons = cdr

def deconsify(cons):
    return list(ideconsify(cons))


ten = consify(range(10))
list(run(
        lambda f, t, other: ALL(
            firsto(ten, f),
            tailo(ten, t),
            firsto(other, f))
    ))

assert deconsify(consify([])) == []
assert deconsify(consify(range(100))) == range(100)

Intermezzo: More minikanren

In [66]:
def conde(*disjs):
    """
    Syntactic sugar around the ANY - ALL template
    
    conde is also a form of if-then-else construct
    conde(
        [if-clause1 then],
        [if-clause2 then],
    )
    """
    return ANY(
        *[ALL(*conjs) for conjs in disjs]
    )

In [67]:
def membero(lst, elt):
    def _f(tail):
        return conde(
            [firsto(lst, elt)],
            [tailo(lst, tail), membero(tail, elt)])
    return fresh(_f)

In [68]:
list(run(
        lambda x: membero(consify(range(12)), 30)
        ))

[]

In [69]:
list(run(
        lambda x: membero(consify(range(12)), 11)
        ))

[(any_value(id=0),)]

In [70]:
list(run(
        lambda x: membero(consify(range(0)), x),
        stop=50))

[]

In [80]:
list(run(
        lambda x: membero(x, 10),
        stop=5))




[([10, any_value(id=2)],),
 ([any_value(id=2), [10, any_value(id=4)]],),
 ([any_value(id=2), [any_value(id=4), [10, any_value(id=6)]]],),
 ([any_value(id=2),
   [any_value(id=4), [any_value(id=6), [10, any_value(id=8)]]]],),
 ([any_value(id=2),
   [any_value(id=4),
    [any_value(id=6), [any_value(id=8), [10, any_value(id=10)]]]]],)]

In [81]:
def appendo(x, y, res):
    """
    concatenate x and y
    """
    def _f(a, b, c):
        return conde(
            # if x is empty, then res is y
            [emptyo(x), equiv(y, res)],
            # otherwise is something like cons(a, b), thus res will be cons(a, c),
            # which means that c is what you get when concatenating b and y
            [conso(a, b, x), conso(a, c, res), appendo(b, y, c) ]) 
    return fresh(_f)

list(run(
        lambda x: appendo(consify([1, 2]), consify([3,4]), x),
        ))

[([1, [2, [3, [4, []]]]],)]

In [82]:
list(run(
        lambda x, y: appendo(x, consify([1, 2]), y),
        stop=3))

[([1, [2, []]], []),
 ([any_value(id=2), [1, [2, []]]], [any_value(id=2), []]),
 ([any_value(id=2), [any_value(id=5), [1, [2, []]]]],
  [any_value(id=2), [any_value(id=5), []]])]

In [111]:
for x, y in run(lambda x, y: appendo(consify(range(3)), y, consify(range(10))), stop=5):
   print x, y

 [3, [4, [5, [6, [7, [8, [9, []]]]]]]] any_value(id=0)


I can count

In [75]:
zero = emptyo
sumo = appendo

def consify_i(n):
    return consify(range(n))

def deconsify_i(c):
    n = 0
    try:
        while c:
            _, c = c
            n += 1
    except:
        return c
    return n

def deconsify_subs(s):
    return {k: deconsify_i(v) for (k,v) in s.iteritems() }

In [76]:
map(deconsify_subs,
    run(
        lambda x, y: conde(
            [sumo(x, y, consify_i(10))],
            ),
        stop=50))

AttributeError: 'tuple' object has no attribute 'iteritems'